In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import get_file

from deepcell.utils.plot_utils import create_rgb_image
from deepcell.datasets import SpotNetExampleData
from deepcell_spots.applications import SpotDecoding, SpotDetection
from deepcell_spots.utils.preprocessing_utils import min_max_normalize
from deepcell_spots.utils.postprocessing_utils import max_cp_array_to_point_list_max
from deepcell_spots.multiplex import extract_spots_prob_from_coords_maxpool

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [2]:
data = SpotNetExampleData()

In [3]:
spots_image, _ = data.load_data(file='MERFISH_example')
spots_image = np.swapaxes(spots_image, 3, 0)
spots_image.shape

(20, 500, 500, 1)

# Spot Detection

This section can be substituted for any other spot detection method. For simplicity, we will use the `SpotDetection` application to perform spot detection.

In [5]:
app = SpotDetection()
app.postprocessing_fn = None

INFO:root:Checking for cached data
INFO:root:Checking SpotDetection-7.tar.gz against provided file_hash...
INFO:root:SpotDetection-7.tar.gz with hash f52d473ad7e4ce33472f1a9a9cae2d85 not found in /root/.deepcell/models
INFO:root:Making request to server
INFO:root:Downloading models/SpotDetection-7.tar.gz with size 18.3 MB to /root/.deepcell/models
18.3MB [00:00, 46.0MB/s]                                                                               
INFO:root:🎉 Successfully downloaded file to /root/.deepcell/models/SpotDetection-7.tar.gz
INFO:root:Extracting /root/.deepcell/models/SpotDetection-7.tar.gz
INFO:root:Successfully extracted /root/.deepcell/models/SpotDetection-7.tar.gz into /root/.deepcell/models
2023-09-04 03:05:15.617537: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operatio

In [6]:
pred = app.predict(spots_image, clip=True, threshold=0.5)

INFO:root:Converting image dtype to float
2023-09-04 03:05:22.760802: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


In [7]:
output_image = pred['classification'][:,...,1:2]
output_image = np.swapaxes(output_image, 0, 3)
output_image.shape

(1, 500, 500, 20)

In [8]:
spots_threshold = 0.9
max_proj_images = np.max(output_image[...,40:-40,40:-40,:20], axis=-1)
spots_locations = max_cp_array_to_point_list_max(max_proj_images,
                                                 threshold=spots_threshold, min_distance=1)

In [9]:
maxpool_extra_pixel_num=0
spots_intensities = extract_spots_prob_from_coords_maxpool(
    output_image[...,40:-40,40:-40,:20], spots_locations, extra_pixel_num=maxpool_extra_pixel_num)
spots_intensities_vec = np.concatenate(spots_intensities)
spots_locations_vec = np.concatenate([np.concatenate(
    [item, [[idx_batch]] * len(item)], axis=1)
    for idx_batch, item in enumerate(spots_locations)])

spots_intensities_vec.shape, spots_locations_vec.shape

((1205, 20), (1205, 3))

# Spot Decoding

In [10]:
rounds = 10
channels = 2
df_barcodes = data.load_data(file='MERFISH_codebook')
df_barcodes

,Gene,RS0015,RS0083,RS0095,RS0109,RS0175,RS0237,RS0247,RS0255,RS0307,...,RS0343,RS0384,RS0406,RS0451,RS0468,RS0548,RS0578,RS0584,RS0639,RS0707
1,Slc51a,0,0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,Maoa,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
3,Hmgb2,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
4,Cps1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,Txndc5,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Blank-20,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0
262,Blank-21,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
263,Blank-22,1,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
264,Blank-23,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
dec_app = SpotDecoding(df_barcodes=df_barcodes,
                       rounds=rounds,
                       channels=channels)

The `SpotDecoding` application takes a vector of spot intensities with the shape (num. spots, `rounds`*`channels`) as its input. 

In [12]:
decoding_result = dec_app.predict(spots_intensities_vec)

Training...


100%|████████████████████████████████████████████████████████████████| 500/500 [00:16<00:00, 30.10it/s]


Estimating barcode probabilities...


100%|███████████████████████████████████████████████████████████████| 266/266 [00:00<00:00, 748.11it/s]


Revising errors...


1205it [00:04, 287.16it/s]

283 of 435 rescue attempts were successful.


`spots_locations_vec` can be used to populate a DataFrame, which resembles the Polaris output.

In [13]:
df = pd.DataFrame()
df[['x', 'y', 'batch_id']] = spots_locations_vec.astype(np.int32)
df['cell_id'] = None
for name, val in decoding_result.items():
    df[name] = val
    
df

,x,y,batch_id,cell_id,spot_index,predicted_id,predicted_name,probability,source
0,259,242,0,None,0,5,Txndc5,1.000000,prediction
1,162,251,0,None,1,54,Stmn1,0.999510,prediction
2,193,175,0,None,2,4,Cps1,0.999948,prediction
3,153,50,0,None,3,4,Cps1,1.000000,prediction
4,77,262,0,None,4,4,Cps1,1.000000,prediction
...,...,...,...,...,...,...,...,...,...
1200,270,354,0,None,1200,267,Unknown,0.837297,prediction
1201,137,318,0,None,1201,266,Background,0.934526,error rescue
1202,82,174,0,None,1202,266,Background,0.981621,error rescue
1203,233,385,0,None,1203,266,Background,0.950382,prediction
